In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np

In [2]:
number_of_tag_barcode = 384
number_of_pcr_index = 384
input_cell_number = 500000
### sort_cell_number determine by the distribution sampling  


In [3]:
#### create normal distribution model, e.g. normal distribution 

def normal_distribution_sampling(input_mean, input_sd): 
    if round(np.random.normal(input_mean,input_sd)) >0 :
        return round(np.random.normal(input_mean,input_sd))
    else : 
        return 0


In [4]:
tag_barcode_name_list = []
for i in range(number_of_tag_barcode):
    tag_barcode_name_list.append("TAG"+str(i))

pcr_index_name_list = []
for i in range(number_of_pcr_index):
    pcr_index_name_list.append("PCR"+str(i))


In [5]:
###
class cell:
    def __init__(self, cellID):
        self.cellID = cellID
    def set_tag_barcode(self, tag_barcode):
        self.tag_barcode = tag_barcode
    def set_pcr_index(self,pcr_index):
        self.pcr_index = pcr_index

    def get_tag_barcode(self):
        return self.tag_barcode
    def get_pcr_index(self):
        return self.pcr_index
    def get_cell_info(self):
        cell_info = self.tag_barcode + "|" + self.pcr_index
        return cell_info
        
def select_randomly_from_list(inputList):
    random_index = random.randrange(len(inputList))
    return inputList[random_index]

In [6]:
### randomly add cell with tag_barcode
cell_dict={}
cellID_count =0
for well in range(len(tag_barcode_name_list)):
    this_well_dict = {}
    ### determine sort cell number here 
    distribution_mean = 1000
    distribution_sd = 100
    sort_cell_number = normal_distribution_sampling(distribution_mean,distribution_sd)
    for i in range(round(input_cell_number/len(tag_barcode_name_list))):
        cellID_count = cellID_count+1
        this_cell = cell(str(cellID_count))
        chosen_tag_barcode = tag_barcode_name_list[well]
        this_cell.set_tag_barcode(chosen_tag_barcode)
        cell_dict[cellID_count] = this_cell
# print(cell_dict)


In [7]:
### randomly sort a number of cell to one of the pcr_index well 

second_cell_dict = {}
for well in range(len(pcr_index_name_list)):
    ### pcr_index_name_list[idx] is selected, sort k number of cell into this well
    ### randomly drawn form the cell_dict 
    this_well_dict = {}
    #### determine sort_cell_number here 

    distribution_mean = 15
    distribution_sd = 10
    sort_cell_number = normal_distribution_sampling(distribution_mean,distribution_sd)
    for k in range(sort_cell_number):
        idx = random.choice(list(cell_dict))
        chosen_cell = cell_dict[idx]
        cell_dict.pop(idx)
        chosen_index =  pcr_index_name_list[well]
        chosen_cell.set_pcr_index(chosen_index)
        
        ### build some info to work with downstream
        cell_information = {"tag_barcode" : chosen_cell.get_tag_barcode(), "pcr_index" : chosen_cell.get_pcr_index(),"barcode_index" : chosen_cell.get_cell_info(),"pcr_well" : well }
        this_well_dict[idx] = cell_information 
    second_cell_dict[well] = this_well_dict.copy()

In [8]:
pooled_cell_dict ={} ### pooled final dict
for well in second_cell_dict:
    cell_in_well_dict = second_cell_dict[well]
    for cell in cell_in_well_dict:
        # print(type(cell_in_well_dict[cell]))
        pooled_cell_dict[cell] = cell_in_well_dict[cell].copy()

In [9]:
### calculate simulation collition rate
import pandas as pd
df = pd.DataFrame.from_dict(pooled_cell_dict,    orient="index")
frequency_of_each_combination = df["barcode_index"].value_counts()
collision_count = 0
unique_count =0
for item in frequency_of_each_combination:
    if item != 1:
        collision_count = collision_count + item
    else: 
        unique_count = unique_count + item
### collision rate as a fraction of total 
collision_rate =  collision_count / (collision_count + unique_count)


In [12]:

total_nuclei = len(df)
recovered_barcode = len(frequency_of_each_combination)
cluster_per_nucleus = round(400000000/recovered_barcode)
print("total nuclei generated: ",total_nuclei)
print("recovered barcodes: ",recovered_barcode)
print("collision rate: ",collision_rate)

total nuclei generated:  5826
recovered barcodes:  5662
collision rate:  0.05526948163405424


TAG38|PCR41    3
TAG28|PCR39    2
TAG69|PCR17    2
TAG59|PCR38    2
TAG54|PCR89    2
              ..
TAG81|PCR95    1
TAG58|PCR12    1
TAG3|PCR9      1
TAG74|PCR92    1
TAG54|PCR30    1
Name: barcode_index, Length: 903, dtype: int64


In [11]:
df

,tag_barcode,pcr_index,barcode_index,pcr_well
52932,TAG40,PCR1,TAG40|PCR1,1
167414,TAG128,PCR1,TAG128|PCR1,1
448514,TAG344,PCR1,TAG344|PCR1,1
9838,TAG7,PCR1,TAG7|PCR1,1
393266,TAG302,PCR1,TAG302|PCR1,1
...,...,...,...,...
369040,TAG283,PCR383,TAG283|PCR383,383
25381,TAG19,PCR383,TAG19|PCR383,383
111023,TAG85,PCR383,TAG85|PCR383,383
68549,TAG52,PCR383,TAG52|PCR383,383
